# 📘 Documentação das Tabelas Gold

Este documento descreve a modelagem das tabelas criadas na camada **Gold** do Data Lake, a partir das tabelas da camada **Silver**.  

---

## 1. Dimensão Clientes
- **Objetivo:** Criar uma dimensão de clientes com dados únicos e padronizados.  
- **Fonte:** `silver.vendas_clientes`  
- **Campos:**
  - `cliente_id`: Identificador único do cliente.  
  - `Nome_cliente_gold`: Nome do cliente.  
  - `Email_cliente_gold`: Email do cliente.  
  - `Cidade_cliente_gold`: Cidade do cliente.  

---

## 2. Dimensão Produtos
- **Objetivo:** Criar uma dimensão de produtos com informações corrigidas e associar preço unitário.  
- **Fonte:** `silver.vendas_produtos` e `silver.vendas_itens__pedidos`  
- **Campos:**
  - `produto_id`: Identificador único do produto.  
  - `Nome_produto`: Nome corrigido do produto.  
  - `Categoria_produto`: Categoria corrigida do produto.  
  - `Preco_unitario_produto`: Preço unitário corrigido do produto.  

---

## 3. Dimensão Pedidos
- **Objetivo:** Criar uma dimensão de pedidos sem trazer o campo `cliente_id` (mantido apenas na tabela fato).  
- **Fonte:** `silver.vendas_pedidos` e `silver.vendas_itens__pedidos`  
- **Campos:**
  - `Pedido_id`: Identificador único do pedido.  
  - `Quantidade_pedido`: Quantidade corrigida do pedido.  
  - `Data_pedido`: Data do pedido.  

---

## 4. Fato Itens Pedido
- **Objetivo:** Criar a tabela fato que relaciona clientes, produtos e pedidos, calculando o valor total.  
- **Fonte:** `silver.vendas_itens__pedidos` e `silver.vendas_pedidos`  
- **Campos:**
  - `itens_id`: Identificador único do item.  
  - `cliente_id`: Identificador do cliente associado ao pedido.  
  - `produto_id`: Identificador do produto.  
  - `valor_total`: Valor total calculado (`quantidade_corrigida * preco_unitario_corrigido`).  

---

## 📊 Resumo da Modelagem
- **Dimensões criadas:** Clientes, Produtos, Pedidos.  
- **Fato criada:** Itens Pedido.  
- **Estratégia:**  
  - Dimensões armazenam atributos descritivos e corrigidos.  
  - Fato concentra métricas e relacionamentos entre dimensões.  
  - O modelo segue o padrão **Star Schema**, facilitando análises no Power BI ou outras ferramentas de BI.  


In [0]:
%sql
CREATE DATABASE IF NOT EXISTS gold;

In [0]:
# %sql
# DROP DATABASE IF EXISTS gold CASCADE;

In [0]:
%sql
--Tabela Dimensão clientes

-- CREATE OR REPLACE TABLE gold.gold_clientes
-- USING DELTA
-- AS
-- SELECT DISTINCT 
--     id AS cliente_id, 
--     nome as Nome_cliente_gold, 
--     email as Email_cliente_gold, 
--     cidade as Cidade_cliente_gold
-- FROM silver.vendas_clientes;

-- # -- --Tabela Dimensão produtos 

-- CREATE OR REPLACE TABLE gold.gold_produtos
-- USING DELTA
-- AS
-- SELECT DISTINCT 
--     r.id AS produto_id, 
--     r.nome_corrigido as Nome_produto, 
--     r.categoria_corrigida as Categoria_produto,
--     i.preco_unitario_corrigido as Preco_unitario_produto

-- FROM silver.vendas_produtos r
-- JOIN silver.vendas_itens__pedidos i
-- ON r.id = i.produto_id;


-- -- Tabela Dimensão pedidos

-- -- Preciso retirar Clientes_id para dimensão silver

-- CREATE OR REPLACE TABLE gold.gold_pedidos
-- USING DELTA
-- AS
-- SELECT 
--     p.id AS Pedido_id,
--     i.quantidade_corrigida AS Quantidade_pedido,
--     p.data AS Data_pedido
-- FROM silver.vendas_pedidos p
-- JOIN silver.vendas_itens__pedidos i
-- ON p.id = i.pedido_id;
 -- Tabela Fato - Itens Pedido


CREATE OR REPLACE TABLE gold.gold_itens_pedido
USING DELTA 
AS
SELECT
    i.id AS itens_id,
    p.cliente_id,
    i.produto_id,
    p.id AS pedido_id,
    CAST(i.quantidade_corrigida * i.preco_unitario_corrigido AS DECIMAL(18,2)) AS valor_total,
    CAST(i.quantidade_corrigida * i.preco_unitario_corrigido AS DECIMAL(18,2)) / i.quantidade_corrigida AS `Ticket_médio`
FROM silver.vendas_itens__pedidos i
JOIN silver.vendas_pedidos p
    ON i.pedido_id = p.id;




In [0]:
# %sql

# drop table gold.gold_clientes;
# drop table gold.gold_produtos;
# drop table gold.gold_pedidos;
# drop table gold.gold_itens_pedido;

In [0]:
%sql
MERGE INTO gold.gold_clientes AS target
USING (
    SELECT 
        id AS cliente_id, 
        nome AS Nome_cliente_gold, 
        email AS Email_cliente_gold, 
        cidade AS Cidade_cliente_gold,
        ROW_NUMBER() OVER (PARTITION BY id ORDER BY nome) AS rn
    FROM silver.vendas_clientes
) AS source
ON target.cliente_id = source.cliente_id
WHEN MATCHED AND source.rn = 1 THEN UPDATE SET
    target.Nome_cliente_gold = source.Nome_cliente_gold,
    target.Email_cliente_gold = source.Email_cliente_gold,
    target.Cidade_cliente_gold = source.Cidade_cliente_gold
WHEN NOT MATCHED AND source.rn = 1 THEN INSERT (
    cliente_id, Nome_cliente_gold, Email_cliente_gold, Cidade_cliente_gold
) VALUES (
    source.cliente_id, source.Nome_cliente_gold, source.Email_cliente_gold, source.Cidade_cliente_gold
);


MERGE INTO gold.gold_produtos AS target
USING (
    SELECT 
        r.id AS produto_id, 
        r.nome_corrigido AS Nome_produto, 
        r.categoria_corrigida AS Categoria_produto,
        i.preco_unitario_corrigido AS Preco_unitario_produto,
        ROW_NUMBER() OVER (PARTITION BY r.id ORDER BY i.preco_unitario_corrigido DESC) AS rn
    FROM silver.vendas_produtos r
    JOIN silver.vendas_itens__pedidos i
        ON r.id = i.produto_id
) AS source
ON target.produto_id = source.produto_id
WHEN MATCHED AND source.rn = 1 THEN UPDATE SET
    target.Nome_produto = source.Nome_produto,
    target.Categoria_produto = source.Categoria_produto,
    target.Preco_unitario_produto = source.Preco_unitario_produto
WHEN NOT MATCHED AND source.rn = 1 THEN INSERT (
    produto_id, Nome_produto, Categoria_produto, Preco_unitario_produto
) VALUES (
    source.produto_id, source.Nome_produto, source.Categoria_produto, source.Preco_unitario_produto
);


MERGE INTO gold.gold_itens_pedido AS target
USING (
    SELECT
        i.id AS itens_id,
        p.cliente_id,
        i.produto_id,
        p.id AS pedido_id,
        CAST(i.quantidade_corrigida * i.preco_unitario_corrigido AS DECIMAL(18,2)) AS valor_total,
        ROW_NUMBER() OVER (PARTITION BY i.id ORDER BY p.data DESC) AS rn
    FROM silver.vendas_itens__pedidos i
    JOIN silver.vendas_pedidos p
        ON i.pedido_id = p.id
) AS source
ON target.itens_id = source.itens_id
WHEN MATCHED AND source.rn = 1 THEN UPDATE SET
    target.cliente_id = source.cliente_id,
    target.produto_id = source.produto_id,
    target.pedido_id = source.pedido_id,
    target.valor_total = source.valor_total
WHEN NOT MATCHED AND source.rn = 1 THEN INSERT (
    itens_id, cliente_id, produto_id, pedido_id, valor_total
) VALUES (
    source.itens_id, source.cliente_id, source.produto_id, source.pedido_id, source.valor_total
);


MERGE INTO gold.gold_pedidos AS target
USING (
    SELECT 
        p.id AS Pedido_id,
        i.quantidade_corrigida AS Quantidade_pedido,
        p.data AS Data_pedido,
        ROW_NUMBER() OVER (PARTITION BY p.id ORDER BY p.data DESC) AS rn
    FROM silver.vendas_pedidos p
    JOIN silver.vendas_itens__pedidos i
        ON p.id = i.pedido_id
) AS source
ON target.Pedido_id = source.Pedido_id
WHEN MATCHED AND source.rn = 1 THEN UPDATE SET
    target.Quantidade_pedido = source.Quantidade_pedido,
    target.Data_pedido = source.Data_pedido
WHEN NOT MATCHED AND source.rn = 1 THEN INSERT (
    Pedido_id, Quantidade_pedido, Data_pedido
) VALUES (
    source.Pedido_id, source.Quantidade_pedido, source.Data_pedido
);


In [0]:
%sql
select * from gold.gold_clientes order by cliente_id ;

In [0]:
%sql

-- SELECT cliente_id FROM gold.gold_clientes ORDER BY cliente_id;
SELECT Pedido_id FROM gold.gold_pedidos;

Necessário posicionar a coluna quantidade e fazer a tabela fato 